In [139]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm
from torchsummary import summary

from concrete import fhe
from sklearn.model_selection import train_test_split
import time

In [140]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [141]:
#hyperparams
no_epochs = 2
batch_size = 32
learning_rate = 0.001

In [142]:
#transform the dataset into tensors normalized range [-1, 1]
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5),(0.5))     
        ])

In [143]:
#data sets downloading and reading
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform
                                        )

test_dataset = torchvision.datasets.MNIST(root='data',
                                        train=False,
                                        download=True,
                                        transform=transform
                                        )


In [144]:
# Extract features (images) and labels from MNIST dataset
mnist_features = train_dataset.data.numpy().reshape(-1, 28, 28)
mnist_labels = train_dataset.targets.numpy()

# Reshape and expand dimensions to match the structure of load_digits dataset
x_train_mnist = np.expand_dims(mnist_features, 1)

# Split the MNIST data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x_train_mnist, mnist_labels,  train_size=100, test_size=100, shuffle=True, random_state=42
)

#x_train = x_train.astype('float64')
print(x_train.dtype)
print(x_train.shape)

#print(x_train)
# plt.imshow(x_train[0,0], cmap='grey')
# plt.show()
# Verify the shapes
print("Shape of x_train from MNIST:", x_train.shape)
print("Shape of x_test from MNIST:", x_test.shape)
print("Shape of y_train from MNIST:", y_train.shape)
print("Shape of y_test from MNIST:", y_test.shape)

uint8
(100, 1, 28, 28)
Shape of x_train from MNIST: (100, 1, 28, 28)
Shape of x_test from MNIST: (100, 1, 28, 28)
Shape of y_train from MNIST: (100,)
Shape of y_test from MNIST: (100,)


In [145]:
configuration = fhe.Configuration(
    comparison_strategy_preference=fhe.ComparisonStrategy.ONE_TLU_PROMOTED,
)


In [187]:

weight = np.array([[2, 1], [3, 2]]).reshape(1, 1, 2, 2)
# print(weight.shape)
selected_weights = np.array(x_train[0, 0, :, :]).reshape(1, 1, 28, 28)
print(selected_weights.shape)

@fhe.compiler({"x": "encrypted"})
def network_function(x):
    conv1 =  fhe.conv(x, selected_weights, kernel_shape=(28,28))
    pool1 = fhe.maxpool(conv1, kernel_shape=(2, 2), strides=(2, 2), dilations=(1, 1))
    conv2 = fhe.conv(conv1, selected_weights)
    print(pool1)
    net_result = pool1
    return net_result

(1, 1, 28, 28)


In [188]:
inputset = [x_train]

circuit = network_function.compile(inputset)

# print(sample)
enc_sampe = circuit.encrypt(x_train)
enc_result = circuit.run(enc_sampe)
dec_result = circuit.decrypt(enc_result)
print(dec_result)

Tracer<output=EncryptedTensor<uint8, shape=(100, 1, 1, 1)>>


RuntimeError: Can't compile: Cannot find crypto parameters